In [1]:
## Imports
import trimesh
import open3d as o3d
import numpy as np
from utils import extract_largest_cavity, extract_cavity_parts ,extract_top_percentage
from utils import show_mesh_dimensions_with_cylinders, visualize_roughness, create_cylinder_between_points
from utils import split_side_and_get_normal_means, calculate_roughness, discrete_mean_curvature_measure_gpu


BOTTOM_THRESHOLD_PERCENTAGE= 0.4
MEAN_CURVATURE_RADİUS= 2
TOOTH_PERCENTAGE = 28.0
CAVITY_PERCENTAGE = 20.0



C:\Users\Arif\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\requests\__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (5.2.0)/charset_normalizer (2.0.12) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [155]:

### Load and compute the mean curvature
# Load the tooth STL model using Trimesh
mesh_trimesh = trimesh.load_mesh(f"StudentTeeth/MasterAlignedReversed.stl")

# Get vertices, faces, and normals
old_vertices = np.array(mesh_trimesh.vertices)
old_faces = np.array(mesh_trimesh.faces)
old_normals = np.array(mesh_trimesh.vertex_normals)

old_tooth_o3d = o3d.geometry.TriangleMesh()
old_tooth_o3d.vertices = o3d.utility.Vector3dVector(old_vertices)
old_tooth_o3d.triangles = o3d.utility.Vector3iVector(old_faces)
old_tooth_o3d.compute_vertex_normals()# Convert full tooth to Open3D mesh

old_tooth_o3d.paint_uniform_color([0.8, 0.8, 0.8])  # light gray
# Visualize with both bounding boxes and the original mesh
# Compute Mean Curvature using Trimesh

TriangleMesh with 16382 points and 32628 triangles.

In [156]:
num_vertices = old_vertices.shape[0]
num_samples = int(num_vertices * 0.2)

# Sort vertices by Z-coordinate (descending order)
sorted_indices = np.argsort(old_vertices[:, 2])[::-1]  # Use `1` for Y-axis or `0` for X-axis
top_indices = sorted_indices[:num_samples]  # Select top 10%

# Extract the top points
top_points = old_vertices[top_indices]

# Create a point cloud object
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(top_points)

# Oriented Bounding Box (OBB) - if you want tighter fit but possibly rotated
tooth_top_obb = pcd.get_oriented_bounding_box()
tooth_top_obb.color = (0, 1, 0)  # Green

R = tooth_top_obb.R              # Rotation matrix (3x3)
center = tooth_top_obb.center    # Center of OBB


T_translate_to_origin = np.eye(4)
T_translate_to_origin[:3, 3] = -center

T_translate_back = np.eye(4)
T_translate_back[:3, 3] = center
# Create rotation matrix in 4x4
T_rotate = np.eye(4)
T_rotate[:3, :3] = R  # your rotation matrix
T_final = T_translate_back @ T_rotate @ T_translate_to_origin

mesh_trimesh.apply_transform(T_final)
#rotate tooth_o3d 

world_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(size=5.0, origin=[0, 0, 5])

vertices = np.array(mesh_trimesh.vertices)
faces = np.array(mesh_trimesh.faces)
normals = np.array(mesh_trimesh.vertex_normals)

tooth_o3d = o3d.geometry.TriangleMesh()
tooth_o3d.vertices = o3d.utility.Vector3dVector(vertices)
tooth_o3d.triangles = o3d.utility.Vector3iVector(faces)
tooth_o3d.compute_vertex_normals()# Convert full tooth to Open3D mesh

tooth_o3d.paint_uniform_color([1, 0.8, 0.8])  # light gray
# Visualize with both bounding boxes and the original mesh

TriangleMesh with 16382 points and 32628 triangles.

In [157]:
mean_curvature = trimesh.curvature.discrete_mean_curvature_measure(mesh_trimesh, mesh_trimesh.vertices, MEAN_CURVATURE_RADİUS)

In [158]:
### kavitenin seçilmesi 
cavity_indices = np.where(mean_curvature < 0.4)[0]  # Select all vertices with negative curvature
largest_cavity_mesh, largest_cavity_indices = extract_largest_cavity(vertices, faces, cavity_indices)
cavity_vertices= np.asarray(largest_cavity_mesh.vertices)

top_tooth_mesh = extract_top_percentage(tooth_o3d , TOOTH_PERCENTAGE) 
top_cavity_mesh = extract_top_percentage(largest_cavity_mesh , CAVITY_PERCENTAGE)
tooth_dimension_cylinder_meshes, tooth_width, tooth_length = show_mesh_dimensions_with_cylinders(top_tooth_mesh)
cavity_dimension_cylinder_meshes, cavity_width, cavity_length = show_mesh_dimensions_with_cylinders(largest_cavity_mesh)


### Kavitenin alt kısmının seçilmesi ve kavite yüksekliğinin hesaplanması
side_bottom, cavity_bottom = extract_cavity_parts(largest_cavity_mesh, BOTTOM_THRESHOLD_PERCENTAGE)
print(cavity_width)
# kavite altının z eksenindeki ortalamasını al
bottom_vertices = np.asarray(cavity_bottom.vertices)
bottom_z_values = bottom_vertices[:, 2]
min_z_mean = np.mean(bottom_z_values) 

# kavite alanının en üstü 
max_z = np.max(cavity_vertices[:, 2])
cavity_depth = max_z - min_z_mean  # Derinlik (Z eksenindeki fark)

outline_indices = np.where((mean_curvature > 3.0))[0]
roughness = calculate_roughness(cavity_bottom)
colored_roughness = visualize_roughness(cavity_bottom,tooth_o3d)

cavity_centroid = np.mean(cavity_vertices, axis=0)
min_z_point = [cavity_centroid[0], cavity_centroid[1], min_z_mean]
max_z_point = [cavity_centroid[0], cavity_centroid[1], max_z]
cavity_depth_mesh = create_cylinder_between_points(min_z_point, max_z_point)

right_mesh, left_mesh, right_normal_mean, left_normal_mean = split_side_and_get_normal_means(side_bottom)

cavity_bottom.compute_vertex_normals()
bottom_normal_mean = np.mean(np.asarray(cavity_bottom.vertex_normals),axis=0)

right_angle = np.dot(right_normal_mean, bottom_normal_mean)
left_angle = np.dot(left_normal_mean, bottom_normal_mean)


8.213839632247854


In [ ]:
o3d.visualization.draw_geometries([ world_frame,tooth_o3d,tooth_top_obb,largest_cavity_mesh],mesh_show_back_face=True)

In [218]:


def get_top_right_edge_midpoint_pcd(obb: o3d.geometry.OrientedBoundingBox ,x ,y ) -> o3d.geometry.PointCloud:
    # OBB'nin köşe noktalarını al
    box_corners = np.asarray(obb.get_box_points())

    top_right_edge_midpoint = (box_corners[x] + box_corners[y]) / 2.0

    # Noktayı PointCloud olarak döndür
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector([top_right_edge_midpoint])
    return top_right_edge_midpoint


def point_to_line_distance(point, line_start, line_end):
    point = np.array(point)
    a = np.array(line_start)
    b = np.array(line_end)

    ab = b - a
    t = np.dot(point - a, ab) / np.dot(ab, ab)
    t = np.clip(t, 0.0, 1.0)  # sadece doğru parçası için

    closest_point = a + t * ab
    distance = np.linalg.norm(point - closest_point)
    
    return distance, closest_point



num_vertices = vertices.shape[0]
start_idx  = int(num_vertices * 0.3)
end_idx  = int(num_vertices * 0.5)

# Sort vertices by Z-coordinate (descending order)
sorted_indices = np.argsort(vertices[:, 2])[::-1]  # Use `1` for Y-axis or `0` for X-axis
interval_indices = sorted_indices[start_idx:end_idx]  # Select top 10%

# Extract the top points
interval_points = vertices[interval_indices]

# Create a point cloud object
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(interval_points)

interval_obb = pcd.get_oriented_bounding_box()
interval_obb.color = (1,0,0)

cavity_obb = largest_cavity_mesh.get_oriented_bounding_box()
cavity_obb.color = (0,0,1)
box_corners = np.asarray(interval_obb.get_box_points()) #3,5 4,6

#mesial ridge_width calculation
mesial_line_start = box_corners[4]
mesial_line_end = box_corners[6]
mesial_middle_point = get_top_right_edge_midpoint_pcd(cavity_obb, 4 ,6)
# other_middle = get_top_right_edge_midpoint_pcd(interval_obb)
mesial_ridge_distance , mesial_ridge_closest_point= point_to_line_distance(mesial_middle_point , mesial_line_start , mesial_line_end)
mesial_ridge_width_mesh = create_cylinder_between_points(mesial_ridge_closest_point,mesial_middle_point)
mesial_ridge_width_mesh.paint_uniform_color([1.0, 0.0, 0.0])  

#distal ridge_width calculation
distal_line_start = box_corners[3]
distal_line_end = box_corners[5]
distal_middle_point = get_top_right_edge_midpoint_pcd(cavity_obb, 3 ,5)
# other_middle = get_top_right_edge_midpoint_pcd(interval_obb)
distal_ridge_distance , distal_ridge_closest_point= point_to_line_distance(distal_middle_point , distal_line_start , distal_line_end)
distal_ridge_width_mesh = create_cylinder_between_points(distal_ridge_closest_point,distal_middle_point)
distal_ridge_width_mesh.paint_uniform_color([0.0, 1.0, 0.0])  


o3d.visualization.draw_geometries([ tooth_o3d ,cavity_obb,interval_obb,distal_ridge_width_mesh,mesial_ridge_width_mesh,world_frame],mesh_show_back_face=True)


2.3483726372318685


In [17]:

b_l_length_ratio = (tooth_width - cavity_width) / tooth_width
m_d_length_ratio = (tooth_length - cavity_length) / tooth_length
# Calculate score
is_right_angle = 0
is_left_angle = 0
is_cavity_depth = 0
is_roughness = 0
is_m_d_length_ratio = 0
is_b_l_length_ratio = 0
is_mesial_marginal_ridge_width_true = 0
is_distal_marginal_ridge_width_true = 0 
is_cavity_width = 0
is_mesial_isthmus_length_true = 0
is_distal_isthmus_length_true = 0
score = 0

#degree ye çevir
right_angle = np.degrees(np.arccos(right_angle))
left_angle = np.degrees(np.arccos(left_angle))

# if mesial_isthmus_length >= 1.5 and mesial_isthmus_length <= 1.99: 
#     score += 10
#     is_mesial_isthmus_length_true = 1 
# elif (1.0 <= mesial_isthmus_length < 1.5) or (2.01 <= mesial_isthmus_length <= 2.5):
#     score += 5
#     is_mesial_isthmus_length_true = 0.5 
# elif mesial_isthmus_length < 1.0 or mesial_isthmus_length > 2.5:
#     is_mesial_isthmus_length_true = 0


# if distal_isthmus_length >= 1.5 and distal_isthmus_length <= 1.99: 
#     score += 10
#     is_distal_isthmus_length_true = 1 
# elif (1.0 <= distal_isthmus_length < 1.5) or (2.01 <= distal_isthmus_length <= 2.5):
#     score += 5
#     is_distal_isthmus_length_true = 0.5 
# elif distal_isthmus_length < 1.0 or distal_isthmus_length > 2.5:
#     is_distal_isthmus_length_true = 0

## mesial marginal
if mesial_marginal_ridge_width>=1.2 and mesial_marginal_ridge_width<=1.6:
    score +=10
    is_mesial_marginal_ridge_width_true = 1
elif (mesial_marginal_ridge_width<1.2 and mesial_marginal_ridge_width>=1.0) or (mesial_marginal_ridge_width>1.6 and mesial_marginal_ridge_width<=2.0):
    score += 5
    is_mesial_marginal_ridge_width_true = 0.5
elif mesial_marginal_ridge_width<1.0 or mesial_marginal_ridge_width>2:
    is_mesial_marginal_ridge_width_true = 0
## distal marginal
if distal_marginal_ridge_width>=1.2 and distal_marginal_ridge_width<=1.6:
    score +=10
    is_distal_marginal_ridge_width_true = 1
elif (distal_marginal_ridge_width<1.2 and distal_marginal_ridge_width>=1.0) or (distal_marginal_ridge_width>1.6 and distal_marginal_ridge_width<=2.0):
    score += 5
    is_distal_marginal_ridge_width_true = 0.5
elif distal_marginal_ridge_width<1.0 or distal_marginal_ridge_width>2:
    is_distal_marginal_ridge_width_true = 0


# Bucco-lingual length ratio grading
if b_l_length_ratio >= 0.35 and b_l_length_ratio <= 0.45:
    score += 10
    is_b_l_length_ratio = 1
elif (b_l_length_ratio >= 0.29 and b_l_length_ratio < 0.34):
    score += 5
    is_b_l_length_ratio = 0.5


# Mesio-distal length ratio grading
if m_d_length_ratio >= 0.65 and m_d_length_ratio <= 0.75:
    score += 10
    is_m_d_length_ratio = 1
elif m_d_length_ratio<0.65 and m_d_length_ratio>=0.6:
    score += 5
    is_m_d_length_ratio = 0.5


is_right_angle = 0
if right_angle >80 :
    score += 10
    is_right_angle = 1
elif right_angle > 70 :
    score +=5
    is_right_angle = 0.5

is_left_angle = 0
if left_angle > 80 :
    score +=10
    is_left_angle = 1
elif left_angle > 70 :
    score +=5
    is_left_angle = 0.5

if cavity_length>=2.7 and cavity_length<=3.3:
    score +=10
    is_cavity_length = 1
elif (cavity_length<=2.69 and cavity_length>=2.5) or (cavity_length>=3.31 and cavity_length<=3.5):
    score += 5
    is_cavity_length = 0.5
elif cavity_length<2.5 or cavity_length>3.5:
    is_cavity_length = 0


if cavity_width>=7.1 and cavity_width<=8.29:
    score +=10
    is_cavity_width = 1
elif cavity_width>=6.6 and cavity_width<=7.00:
    is_cavity_width = 0.5
    score += 5
    

if cavity_depth>=2.5 and cavity_depth<=3.0:
    score +=10
    is_cavity_depth = 1
elif (cavity_depth<=2.49 and cavity_depth>=2.0) or (cavity_depth>=3.01 and cavity_depth<=3.49):
    score += 5
    is_cavity_depth = 0.5
elif cavity_depth<2.0 or cavity_depth>3.5:
    is_cavity_depth = 0

std_roughness = np.std(roughness)

if std_roughness>=0 and std_roughness<=10.0:
    score +=10
    is_roughness = 1
elif std_roughness>=10.01 and std_roughness<=40.00:
    score += 5
    is_roughness = 0.5
elif std_roughness>40.00:
    is_roughness = 0

In [18]:
data = {
    # "mesial_isthmus_length": round(mesial_isthmus_length, 3),
    # "is_mesial_isthmus_length_true": is_mesial_isthmus_length_true,

    # "distal_isthmus_length": round(distal_isthmus_length, 3),
    # "is_distal_isthmus_length_true": is_distal_isthmus_length_true,

    "right_angle": round(right_angle,3),
    "is_right_angle_true" : is_right_angle, 

    "left_angle": round(left_angle,3),
    "is_left_angle_true" : is_left_angle , 
    
    "cavity_depth": round(cavity_depth,3),
    "is_cavity_depth_true" : is_cavity_depth ,  
    
    "roughness":  round(np.std(roughness),3),
    "is_roughness_true" : is_roughness , 
    
    "m_d_length_ratio" : round(m_d_length_ratio,3) ,
    "is_m_d_length_ratio_true" : is_m_d_length_ratio ,
    
    "m_d_length" : round(cavity_width,3) ,
    "is_m_d_length_true" : is_cavity_width ,

    "b_l_length_ratio" : round(b_l_length_ratio,3),
    "is_b_l_length_ratio_true" : is_b_l_length_ratio ,
    
    "b_l_length" : round(cavity_length,3),
    "is_b_l_length_true" : is_cavity_length , 

    "distal_marginal_ridge_width" : round(distal_marginal_ridge_width,3) ,
    "is_distal_marginal_ridge_width_true" : is_distal_marginal_ridge_width_true,
    
    "mesial_marginal_ridge_width" : round(mesial_marginal_ridge_width,3) ,
    "is_mesial_marginal_ridge_width_true" : is_mesial_marginal_ridge_width_true ,

    "score" : score
}


In [19]:
data

{'right_angle': 82.626,
 'is_right_angle_true': 1,
 'left_angle': 83.63,
 'is_left_angle_true': 1,
 'cavity_depth': 2.02,
 'is_cavity_depth_true': 0.5,
 'roughness': 4.204,
 'is_roughness_true': 1,
 'm_d_length_ratio': 0.421,
 'is_m_d_length_ratio_true': 0,
 'm_d_length': 6.123,
 'is_m_d_length_true': 0,
 'b_l_length_ratio': 0.457,
 'is_b_l_length_ratio_true': 0,
 'b_l_length': 6.921,
 'is_b_l_length_true': 0,
 'distal_marginal_ridge_width': 3.476,
 'is_distal_marginal_ridge_width_true': 0,
 'mesial_marginal_ridge_width': 3.644,
 'is_mesial_marginal_ridge_width_true': 0,
 'score': 35}